# CWL Execution - *Workflow* Class 

link to documentation: https://eoap.github.io/quickwin/cwl-workflow/cloud-native/

## CWL Inspection
Add the `Workflow` Class on top of the `CommandLineTool` Class, in order to build up the full Application workflow

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e . -

There's a new element `$graph`, it's there to list all classes of the CWL document. In our case, one `Workflow` and one `CommandLineTool`:

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[].class' - 

Let's inspect the `Workflow` element keys:

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0] | keys' - 

The Workflow must have an `id`, a `label` and a `doc`:

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0] | {"id": .id, "label": .label, "doc": .doc }' - 

The `inputs` mapping defines all the workflow inputs. 

These are then mapped to the CommandLineTool in the `step` mapping

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].inputs' - 

The `steps` mapping defines a single step `node_detect` that runs the CWL element with id `detect-water-body`

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].steps' - 

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].steps.node_detect.run' - 

The mapping between the Workflow class inputs and the CommandLineTool inputs is done in the `in` dictionary: 

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].steps.node_detect.in' - 

Finally the outputs are defined with:

- an `id` 
- an `outputSource` that is linked to the CommandLineTool output
- a `type`, the same as the CommandLineTool output type

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[0].outputs' - 

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[1].outputs' - 

In [ ]:
cat ../cwl-workflow/app-water-body-cloud-native.cwl | yq e .'$graph[1].outputs.stac-catalog.type' - 

## CWL Execution

Let's use a CWL runner to run this CWL document.

First create the `params.yaml` file:


In [1]:
cat << EOF > params.yaml
item: https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
aoi: "-121.399,39.834,-120.74,40.472"
epsg: "EPSG:4326"
band: 
- green
- nir
EOF

Then use the `cwltool` to run the CWL document with the parameters:

In [4]:
cwltool ../cwl-workflow/app-water-body-cloud-native.cwl params.yaml > output.json

INFO /Users/simonevaccari/miniconda3/bin/cwltool 3.1.20210628163208
INFO Resolved '../cwl-workflow/app-water-body-cloud-native.cwl' to 'file:///Users/simonevaccari/Documents/repos/eoap/quickwin/cwl-workflow/app-water-body-cloud-native.cwl'
INFO [workflow ] start
INFO [workflow ] starting step node_detect
INFO [step node_detect] start
INFO ['docker', 'pull', 'ghcr.io/eoap/quickwin/detect-water-body:1.0.0']
1.0.0: Pulling from eoap/quickwin/detect-water-body

3ee49805: Pulling fs layer 
0daa4c1f: Pulling fs layer 
269b22c3: Pulling fs layer 
580aad05: Pulling fs layer 
87594817: Pulling fs layer 
Digest: sha256:56eb456782a609d3dc861789e1108fb0e363462e5c1612d51da47b693931c6b1
Status: Downloaded newer image for ghcr.io/eoap/quickwin/detect-water-body:1.0.0
ghcr.io/eoap/quickwin/detect-water-body:1.0.0
WARNING [job node_detect] Skipping Docker software container '--memory' limit despite presence of ResourceRequirement with ramMin and/or ramMax setting. Consider running with --strict-memory-

Inspect the `output.json` file keys:

In [ ]:
cat output.json | yq e '. | keys' - 

Now get the root folder of the results:

In [ ]:
cat output.json | yq e '.stac_catalog.location' -

We know there's a catalog.json file there:


In [ ]:
stac describe $( cat output.json | yq e '.stac_catalog.location' - | sed 's|file://||g' )/catalog.json